In [1]:
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from difflib import SequenceMatcher

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
attributes=pd.read_csv('attributes.csv')

In [9]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [3]:
attributes=attributes.dropna(how='all')
attributes.reset_index(inplace=True)

In [4]:
Trans_Attr=pd.DataFrame(None,index=attributes.product_uid.unique(),columns=attributes.groupby('name').count().index)

In [6]:
Trans_Attr.shape

(86263, 5410)

In [ ]:
for i in range(len(attributes)):
    Trans_Attr[attributes.name[i]].loc[attributes.product_uid[i]]=attributes.value[i]

In [41]:
len(attributes.product_uid.unique())

86264

In [250]:
All=pd.merge(Trans_Attr,train,left_index=True,right_on='product_uid')

In [248]:
def match(a,b):
    if isinstance(a,basestring)==False:
        return 0
    else:
        return SequenceMatcher(None,a,b).ratio()

In [275]:
All2=pd.DataFrame(None,columns=All.columns)

In [293]:
for i in range(1,len(All)):
    All2=All2.append(All.iloc[i].apply(lambda x: match(x,All.search_term.iloc[i])))

In [272]:
All.shape

(56547, 5415)

In [304]:
All2.head()

name,# of Line Wires,# of Rails,# of Vertical Braces,# of Vertical Spacing Sizes,15 Gauge Finish Nailer Included,16 Gauge Finish Nailer Included,18 Gauge Finish Stapler Included,2-Way Intercom Camera,2-piece drawbar,3-Axis Adjustable Camera,...,Wrench Size,Wrench length (in.),Wrench type,Wrinkle resistant,Yarn Gauge (in.),Yaw Adjustment,Zippered Top,Zone-specific Sounds,product_title,relevance
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217391,3.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095238,2.50
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159091,3.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212766,2.33
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.312500,2.67


In [305]:
import statsmodels.api as sm

In [309]:
model=sm.OLS(All2['relevance'],All2.drop(['relevance'],axis=1))

In [311]:
result=model.fit()

In [322]:
print result.summary()

                            OLS Regression Results                            
Dep. Variable:              relevance   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                     202.7
Date:                Thu, 23 Mar 2017   Prob (F-statistic):               0.00
Time:                        14:38:23   Log-Likelihood:                -50207.
No. Observations:               56547   AIC:                         1.085e+05
Df Residuals:                   52497   BIC:                         1.447e+05
Df Model:                        4050                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------

In [326]:
All2.columns[result.pvalues<=0.05]

Index([u'90 Degree Crosscut Capacity Maximum - Left',
       u'90 Degree Crosscut Capacity Maximum - Right', u'Accessory Type',
       u'Accessory or kit type', u'Accessory type', u'Accessory/Part Type',
       u'Actual Product Height (in.)', u'Actual Product Width (in.)',
       u'Actual post width (in.)', u'Actual product width (in.)',
       ...
       u'Wire Type', u'Wire-Break Alarm', u'Wired or Wireless',
       u'Wireless Cameras', u'Wiring Configuration', u'Wood Veneer',
       u'Work Light', u'Work light included',
       u'Works with all water softeners', u'product_title'],
      dtype='object', name=u'name', length=804)

In [329]:
test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [339]:
SequenceMatcher(None,,'Banana').ratio()

0.7058823529411765

In [342]:
test1=test[0:1000]

In [343]:
test1.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [349]:
test_matrix=pd.merge(Trans_Attr,test1,left_index=True,right_on='product_uid')

In [352]:
test_result=pd.DataFrame(columns=test_matrix.columns)

In [353]:
for i in range(1,len(test_matrix)):
    test_result=test_result.append(test_matrix.iloc[i].apply(lambda x: match(x,test_matrix.search_term.iloc[i])))

In [358]:
test_result=test_result.drop(['search_term'],axis=1)

In [360]:
result.predict(test_result)

array([  0.95601667,   2.04657467,   2.60667553,   2.48847485,
         1.29818928,   2.43962028,   1.96570176,   1.89888416,
         2.36083559,   2.68881161,   2.91899649,   2.09410579,
         2.71895088,   2.49232978,   2.6082241 ,   2.57589731,
         2.25904808,   2.36107632,   2.38319538,   2.59165583,
         1.77682863,   2.32181164,   2.67951405,   2.26237688,
         2.12398936,   2.14892134,   2.02143311,   3.19644692,
         2.43499189,   2.32319087,   2.0738686 ,   2.33423621,
         2.28234414,   2.78144434,   1.90636816,   2.11093397,
         2.10385305,   2.46212824,   2.4496613 ,   2.05077784,
         2.00591798,   1.65066874,   1.81936393,   1.33953159,
         2.17800128,   2.67369186,   2.76734053,   2.22179077,
         2.66463156,   1.77214853,   1.94146299,   1.9724114 ,
         1.94887273,   1.22280089,   2.43704119,   2.39539442,
         2.38867824,   1.75801828,   1.89438369,   2.46838423,
         2.63123242,   2.1596882 ,   2.5441577 ,   0.60

In [367]:
len(test.product_uid.unique())

97460

In [ ]:
!conda update scikit-learn

In [380]:
from sklearn import model_selection

ImportError: cannot import name model_selection

In [ ]:
1+1